***⚠️ This project is for learning purposes only.  
The SQLite `.db` file used here was created as part of a personal SQL practice exercise and does not represent real IMDb data.***


# 🧠SQL Mini Project : Analysis of IMDb Movie Data
This notebook explores a mock IMDb movie dataset using SQL. 
👉Following are the concepts practiced:
 - `JOIN` Operations
 - Aggregation with `GROUP BY`
 - `RANK()`
 - Filtering with subqueries
   

## 💡Key Insights include:
- Top 3 rated movies as per genre, ratings
- Average ratings per decade and highest average
- Most voted movies per year


## ✅ Summary
- Sci-Fi and Action are the most common genres
- The 2010s had the highest-rated movies on average
- Interstellar and Inception consistently rank among the top by both rating and votes





In [ ]:
#Run this cell first
import sqlite3
import pandas as pd
conn = sqlite3.connect('/kaggle/input/imdb-sql-project/imdb_project.db')
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:")
print(tables)

#To simply view the three tables
print("🎬 Movies table")
display(pd.read_sql("SELECT * FROM movies LIMIT 10;", conn))
print("⭐ Ratings table")
display(pd.read_sql("SELECT * FROM ratings LIMIT 10;",conn))
print("🏷️ Genres table")
display(pd.read_sql("SELECT * FROM genres LIMIT 10;",conn))


In [ ]:
#Join movies and ratings
query1='''
        SELECT m.title,m.year,r.rating,r.votes
        FROM movies as m
        JOIN ratings as r 
          ON m.id=r.movie_id
      '''
results1 = pd.read_sql(query1, conn)
print("🎬 Movies with Ratings:")
display(results1.head())

In [ ]:
#Top 3 movies by ratings
query2='''
       SELECT m.title ,g.genre_name, r.rating
       FROM movies m
       JOIN genres g 
           ON m.genre_id=g.genre_id
       JOIN ratings r 
           ON m.id=r.movie_id
       ORDER BY r.rating DESC
       LIMIT 3;
       '''
results2=pd.read_sql(query2, conn)
print("⭐Top 3 most rated movies:")
display(results2.head())


In [ ]:
#Average rating per decade
query3='''
       SELECT (m.year/10)*10 AS decade, ROUND(AVG(r.rating),2) AS avg_rating
       FROM movies m 
       JOIN ratings r 
            ON m.id=r.movie_id
       GROUP BY decade
       ORDER BY decade
       '''
results3=pd.read_sql(query3, conn)
print("🧮Average Rating per Decade:")
display(results3.head())


In [ ]:
#Top 3 movies as per genre
query4 = '''
         SELECT *
         FROM (
               SELECT 
               g.genre_name AS genre,
               m.title,
               r.rating,
               RANK() OVER (PARTITION BY g.genre_name ORDER BY r.rating DESC) AS rank
              FROM movies m
              JOIN ratings r ON m.id = r.movie_id
              JOIN genres g ON m.genre_id = g.genre_id
              ) sub
              WHERE rank <= 3;
        '''
results4 = pd.read_sql(query4, conn)
print("⭐ Top 3 rated movies per genre:")
display(results4)

In [ ]:
#Find most voted movie each year
query5='''
       SELECT title, year, votes
       FROM(
            SELECT title, year, votes,
                 RANK() OVER(PARTITION BY year ORDER BY votes DESC) AS vote_rank
            FROM movies m
            JOIN ratings r 
                 ON m.id=r.movie_id
            )sub
            WHERE vote_rank=1;
      '''
results5=pd.read_sql(query5,conn)
print("✨Most Voted Movie Each Year:")
display(results5)

In [ ]:
#Most Common Genre
query6='''
       SELECT g.genre_name, COUNT(*) AS movie_count
       FROM movies m
       JOIN genres g 
            ON m.genre_id=g.genre_id
       GROUP BY g.genre_name
       ORDER BY movie_count DESC
       LIMIT 1;
       '''
results6=pd.read_sql(query6,conn)
print("🏆Most Common Genre:")
display(results6.head())


In [ ]:
#Highest average rated genre
query7='''
       SELECT g.genre_name, ROUND(AVG(r.rating),2) AS avg_rating
       FROM movies m
       JOIN genres g ON g.genre_id=m.genre_id
       JOIN ratings r ON r.movie_id=m.id
       GROUP BY g.genre_name
       ORDER BY avg_rating DESC
       LIMIT 1;
       '''
results7=pd.read_sql(query7, conn)
print("📈Highest Average Rated Genre:")
display(results7.head())